In [2]:
import fasttext
import logging
import numpy as np

In [2]:
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 数据处理

处理百度数据为fasttxt可接收格式

## 非知识点多标签数据

In [3]:
label2id = eval(open('./data/kkb/label_id_19.json', 'r').readlines()[0])
id2label = {v:k for k,v in label2id.items()}
label2id

{'高中_地理': 0,
 '高中_历史': 1,
 '高中_生物': 2,
 '高中_政治': 3,
 '高中_政治_经济学常识': 4,
 '高中_政治_科学社会主义常识': 5,
 '高中_政治_公民道德与伦理常识': 6,
 '高中_历史_古代史': 7,
 '高中_历史_近代史': 8,
 '高中_历史_现代史': 9,
 '高中_地理_人口与城市': 10,
 '高中_地理_生产活动与地域联系': 11,
 '高中_地理_宇宙中的地球': 12,
 '高中_生物_分子与细胞': 13,
 '高中_生物_遗传与进化': 14,
 '高中_生物_稳态与环境': 15,
 '高中_生物_生物技术实践': 16,
 '高中_生物_生物科学与社会': 17,
 '高中_生物_现代生物技术专题': 18}

In [4]:
# Baidu 19 class
x = open('./data/kkb/x_19.txt').readlines()
y = eval(open('./data/kkb/y_19.txt').readlines()[0])

prefix = '__label__'
data = ''

for i in range(len(y)):
    labels = id2label[y[i][1]].split('_')
    labels = [prefix+label for label in labels]
    data += ' '.join(labels[1:]) + ' ' + x[i]


data = data.split('\n')
np.random.shuffle(data)
with open('./data/kkb/baidu_19.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data))

切分数据集

In [10]:
# !head -n 19151 ./data/kkb/baidu_19.txt > ./data/kkb/baidu_19.train
# !tail -n 3000 ./data/kkb/baidu_19.txt > ./data/kkb/baidu_19.valid

## 知识点多标签数据

切分数据集

In [29]:
# !head -n 19576 ./data/kkb/fasttext/baidu_95__label__.txt > ./data/kkb/fasttext/baidu_95__label__.train
# !tail -n 3000 ./data/kkb/fasttext/baidu_95__label__.txt > ./data/kkb/fasttext/baidu_95__label__.valid

# 训练并验证

In [30]:
train_file = r'./data/kkb/fasttext/baidu_95__label__.train'
valid_file = r'./data/kkb/fasttext/baidu_95__label__.valid'

In [31]:
def f1_np(y_true, y_pred):
    """F1 metric.
    
    Computes the micro_f1 and macro_f1, metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=0)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=0)
    
    """Macro_F1 metric.
    """
    precision = true_positives/(predicted_positives+1e-8)
    recall = true_positives/(possible_positives+1e-8)
    macro_f1 = np.mean(2*precision*recall/(precision+recall+1e-8))
        
    """Micro_F1 metric.
    """
    precision = np.sum(true_positives)/np.sum(predicted_positives)
    recall = np.sum(true_positives)/np.sum(possible_positives)
    micro_f1 = 2*precision*recall/(precision+recall+1e-8)
    return micro_f1, macro_f1
    

y_true = np.array([[1,0,1,1,0],[1,1,0,1,1]])
y_pred = np.array([[0,1,1,1,0],[1,1,1,0,1]])

f1_np(y_true, y_pred)

(0.7142857092857143, 0.7333333232222222)

训练模型  
`help(fasttext)`,`help(model)`

In [32]:
model = fasttext.train_supervised(input=train_file, epoch=1000, wordNgrams=5, bucket=200000, dim=50, loss='ova')

验证模型

In [33]:
label_true, label_pred = [], []
with open(valid_file) as f:
    for line in f.readlines():
        labels = line.split()[:-1]
        string = line.split()[-1]
        predicts = model.predict(string, k=2)
        label_true.append(set(labels))
        label_pred.append(set(predicts[0]))

评估模型

In [34]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(label_true)
y_pred = mlb.transform(label_pred)
f1_np(y_true, y_pred)

(0.4199475028119923, 0.23017252353009304)

结果  

|标签|类别|icro_f1|macro_f1|参数|
|:--:|:--:|:--:|:--:|:--:|
|非知识点baidu | 19 | 0.424 | 0.206| - |
|知识点baidu | 95 | 0.421 | 0.234| # epoch=1000, wordNgrams=5, bucket=200000, dim=50|

保存模型

In [35]:
model.save_model("./model/model_baidu_95.bin")